In [ ]:
# Cài đặt nvm
curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.39.1/install.sh | bash

# Thêm nvm vào shell profile
echo 'export NVM_DIR="$([ -z "${XDG_CONFIG_HOME-}" ] && printf %s "${HOME}/.nvm" || printf %s "${XDG_CONFIG_HOME}/nvm")"' >> ~/.bashrc
echo '[ -s "$NVM_DIR/nvm.sh" ] && \. "$NVM_DIR/nvm.sh"' >> ~/.bashrc
echo '[ -s "$NVM_DIR/bash_completion" ] && \. "$NVM_DIR/bash_completion"' >> ~/.bashrc
source ~/.bashrc

# Cài đặt và sử dụng phiên bản Node.js tương thích (ví dụ: phiên bản 14)
nvm install 14

unset NPM_CONFIG_PREFIX
nvm use 14

In [ ]:
!mkdir tea-api
!cd myapi
!npm init -y
!touch server.js
!npm install mysql2
npm i express
mkdir routers
mkdir controllers
mkdir models

In [1]:
import pymysql

# Thiết lập kết nối đến cơ sở dữ liệu MySQL
conn = pymysql.connect(host='localhost', port=3306, user='root', database='shop')

# Tạo đối tượng cursor
cursor = conn.cursor()
listdata = {}

# Lấy danh sách bảng trong cơ sở dữ liệu
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()

# Lấy danh sách cột trong mỗi bảng
for table in tables:
    cursor.execute(f"SHOW COLUMNS FROM {table[0]}")
    columns = cursor.fetchall()
    tam = []
    for column in columns:
        tam.append(column[0])
    listdata[table[0]] = {'columns': tam, 'relationships': []}

# Lấy thông tin về các khóa ngoại từ information_schema
cursor.execute("""
    SELECT 
        TABLE_NAME, 
        COLUMN_NAME, 
        REFERENCED_TABLE_NAME, 
        REFERENCED_COLUMN_NAME
    FROM 
        information_schema.KEY_COLUMN_USAGE
    WHERE 
        CONSTRAINT_SCHEMA = 'shop' AND 
        REFERENCED_TABLE_NAME IS NOT NULL
""")
foreign_keys = cursor.fetchall()

# Thêm thông tin mối quan hệ vào listdata
for fk in foreign_keys:
    table_name = fk[0]
    column_name = fk[1]
    referenced_table_name = fk[2]
    referenced_column_name = fk[3]
    relationship = {
        'column': column_name,
        'referenced_table': referenced_table_name,
        'referenced_column': referenced_column_name
    }
    listdata[table_name]['relationships'].append(relationship)

# Đóng kết nối
cursor.close()
conn.close()

listdata


{'categories': {'columns': ['category_id', 'name', 'image'],
  'relationships': []},
 'customers': {'columns': ['customer_id',
   'customer_name',
   'email',
   'address',
   'phone',
   'password',
   'search'],
  'relationships': []},
 'order_details': {'columns': ['order_detail_id',
   'order_id',
   'product_id',
   'quantity',
   'price'],
  'relationships': [{'column': 'order_id',
    'referenced_table': 'orders',
    'referenced_column': 'order_id'},
   {'column': 'product_id',
    'referenced_table': 'products',
    'referenced_column': 'product_id'}]},
 'orders': {'columns': ['order_id', 'customer_id', 'order_date', 'total'],
  'relationships': [{'column': 'customer_id',
    'referenced_table': 'customers',
    'referenced_column': 'customer_id'}]},
 'product_related': {'columns': ['product_id', 'product_related_id'],
  'relationships': [{'column': 'product_id',
    'referenced_table': 'products',
    'referenced_column': 'product_id'},
   {'column': 'product_related_id',
   

In [2]:
routeLink="/home/ubuntu/shopapi/routers/"
modelLink="/home/ubuntu/shopapi/models/"
controllerLink="/home/ubuntu/shopapi/controllers/"
serverLink="/home/ubuntu/shopapi/server.js"

In [7]:
stri="""
const mysql = require('mysql2');

const connection = mysql.createConnection({
  host: 'localhost',
  user: 'root',
  password: '',
  database: 'shop'
});

connection.connect((err) => {
  if (err) {
    console.error('Error connecting to the database:', err);
    console.log('vua ket noi')
    return;
  }
  console.log('Connected to the MySQL database');
});

function reconnect(){
    connection.connect((err) => {
      if (err) {
        console.error('Error connecting to the database:', err);
        console.log('vua ket noi')
        return;
      }
      console.log('Connected to the MySQL database');
    });
}
//setInterval(reconnect,50000)
module.exports = connection;
"""
with open(modelLink+"connect.js", 'w') as f:
    f.write(stri)

In [7]:
#i=ten bang
#j["columns"]=danh sach cot
#j["relationships"][0]["column"]= cot lien ket
#j["relationships"][0]["referenced_table"]= bang lien ket
for i,j in listdata.items():
    tb=i
    dscot=j["columns"]
    mr=""
    if(j["relationships"]):
        clk=j["relationships"][0]["column"]
        blk=j["relationships"][0]["referenced_table"]
        mr="left join "+blk+" on "+blk+"."+clk+" = "+tb+"."+clk
    stri="""const connection=require("./connect.js");

const create{table} = ({table}) => {{
    return new Promise((resolve,reject)=>{{
        const sql = "INSERT INTO {table}({dscot}) VALUES({dshc})";
        connection.query(sql, [{dst}], (err, result) => {{
            if (err || result.insertId==0) {{
                console.error('Error creating {table}:', err);
                reject({{message:"failed"}});
            }}
            resolve(result.insertId);
        }});
    }});
}};

// Đọc dữ liệu
const get{table} = () => {{
    return new Promise((resolve, reject) => {{
        const sql = 'SELECT {table}.{id},{dscot} FROM {table} {mr}';
        connection.query(sql, (err, result) => {{
            if (err) {{
                console.error('Error fetching {table}:', err);
                reject({{message:"failed"}});
            }} else {{
                resolve(result); // Chuyển đổi kết quả thành chuỗi JSON
            }}
        }});
    }});
}};

// Cập nhật dữ liệu
const update{table} = ({table},id) => {{
    return new Promise((resolve,reject)=>{{
        const sql = 'UPDATE {table} SET {dss} WHERE {table}.{id} = ?';
        connection.query(sql, [{dst}, id], (err, result) => {{
            if (err || result.affectedRows==0) {{
                console.error('Error updating {table}:', err);
                reject({{message:"failed"}});
            }}
            resolve({{message:"ok"}});
        }});
    }});
}};

// Xóa dữ liệu
const delete{table} = (id) => {{
    return new Promise((resolve,reject)=>{{
        const sql = 'DELETE FROM {table} WHERE {table}.{id} = ?';
        connection.query(sql, [id], (err, result) => {{
            if (err || result.affectedRows==0) {{
                console.error('Error deleting:', err);
                reject({{message:"failed"}});
            }}
            resolve({{message:"ok"}});
        }});
    }});
}};

const one{table} = (id) => {{
    return new Promise((resolve, reject) => {{
        const sql = 'SELECT {table}.{id},{dscot} FROM {table} {mr} WHERE {table}.{id} = ?';
        connection.query(sql, [id], (err, result) => {{
            if (err) {{
                console.error('Error fetching {table}:', err);
                reject({{message:"failed"}});
            }} else {{
                resolve(result); // Chuyển đổi kết quả thành chuỗi JSON
            }}
        }});
    }});
}};

const by{table} = (id) => {{
    return new Promise((resolve, reject) => {{
        const sql = 'SELECT {table}.{id},{dscot} FROM {table} {mr} WHERE {table}.name LIKE ?';
        connection.query(sql, ['%'+id+'%'], (err, result) => {{
            if (err) {{
                console.error('Error fetching {table}:', err);
                reject({{message:"failed"}});
            }} else {{
                resolve(result); // Chuyển đổi kết quả thành chuỗi JSON
            }}
        }});
    }});
}};

module.exports = {{ create{table}, get{table}, update{table}, delete{table}, one{table}, by{table} }};
    """.format(
                table=tb,
               id=dscot[0],
              dscot=",".join([tb+"."+item for item in dscot[1:]]),
               dshc=",".join(['?']*(len(dscot)-1)),
               dst=",".join([tb+"."+item for item in dscot[1:]]),
               dss=",".join([tb+"."+item+"=?" for item in dscot[1:]]),
                mr=mr
              )
    with open(modelLink+tb+"Model.js", 'w') as f:
        f.write(stri)

In [6]:
for i, j in listdata.items():
    stri = """const {{ create{table}, get{table}, update{table}, delete{table}, one{table}, by{table} }} = require("../models/{table}Model")
//@desc Get all {tableu}
//@route GET /{tableu}
//@access public
const get{tableu} = async (req, res) => {{
    const data = await get{table}();
    if (data.length > 0) {{
        res.status(200).json(data);
    }} else {{
        res.status(400).json({{ message: "failed" }});
    }}
}}

//@desc Create {tableu}
//@route POST /{tableu}
//@access public
const create{tableu} = async (req, res) => {{
    try {{
        const data = await create{table}(req.body);
        res.status(200).json(data);
    }} catch (error) {{
        res.status(400).json({{ message: "failed" }});
    }}
}}

//@desc Update {tableu}
//@route PUT /{tableu}/id
//@access public
const update{tableu} = async (req, res) =>{{
    try {{
        const data = await update{table}(req.body,req.params.id);
        res.status(200).json(data);
    }} catch (error) {{
        res.status(400).json({{ message: "failed" }});
    }}
}}

//@desc Delete {tableu}
//@route DELETE /{tableu}/id
//@access public
const delete{tableu} = async (req, res) => {{
    try{{
        const data = await delete{table}(req.params.id);
        res.status(200).json(data);
    }}catch (error){{
        res.status(400).json({{ message: "failed" }});
    }}
}}

//@desc Get one {tableu}
//@route GET /{tableu}/id
//@access public
const one{tableu} = async (req, res) => {{
    const data = await one{table}(req.params.id);
    if (data.length > 0) {{
        res.status(200).json(data);
    }} else {{
        res.status(400).json({{ message: "failed" }});
    }}
}}

//@desc Get by {tableu}
//@route GET /{tableu}/by/id
//@access public
const by{tableu} = async (req, res) => {{
    const data = await by{table}(req.params.id);
    if (data.length > 0) {{
        res.status(200).json(data);
    }} else {{
        res.status(400).json({{message: "failed"}});
    }}
}}

module.exports = {{ get{tableu}, create{tableu}, update{tableu}, delete{tableu}, one{tableu}, by{tableu} }};
""".format(table=i, tableu=i.capitalize())

    with open(controllerLink+i+"Controller.js", 'w') as f:
        f.write(stri)


In [4]:
for i,j in listdata.items():
    stri="""const express=require("express")
const {{get{table},create{table},update{table},delete{table},one{table},by{table}}}=require("../controllers/{tableu}Controller")
const router=express.Router()

router.route("/").get(get{table})
router.route("/").post(create{table})
router.route("/:id").put(update{table})
router.route("/:id").delete(delete{table})
router.route("/:id").get(one{table})
router.route("/by/:id").get(by{table})

module.exports = router""".format(table=i.capitalize(),tableu=i)
    with open(routeLink+i+"Route.js", 'w') as f:
        f.write(stri)

In [52]:
stri=""
for i,j in listdata.items():
    stri+='\napp.use("/'+i+'",require("./routers/'+i+'Route"));'
print(stri)


app.use("/categories",require("./routers/categoriesRoute"));
app.use("/customers",require("./routers/customersRoute"));
app.use("/order_details",require("./routers/order_detailsRoute"));
app.use("/orders",require("./routers/ordersRoute"));
app.use("/product_related",require("./routers/product_relatedRoute"));
app.use("/product_reviews",require("./routers/product_reviewsRoute"));
app.use("/product_sales",require("./routers/product_salesRoute"));
app.use("/products",require("./routers/productsRoute"));
app.use("/purchase_details",require("./routers/purchase_detailsRoute"));
app.use("/purchases",require("./routers/purchasesRoute"));
app.use("/sales",require("./routers/salesRoute"));
app.use("/suppliers",require("./routers/suppliersRoute"));


In [53]:
striserver="""const express =require('express')
const cors = require('cors');
const app=express()

app.use(cors()); // Sử dụng middleware CORS

app.use(express.json())
app.use("/category",require("./routers/categoryRoute"));
"""+stri+"""

app.listen(5000,()=>{
    console.log("Lank")
})
"""
with open(serverLink, 'w') as f:
    f.write(striserver)